In [1]:
import pandas as pd
import numpy as np
from string import punctuation
import random
import math
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.corpus import stopwords
from nltk.corpus import movie_reviews
from nltk import word_tokenize
from textblob import TextBlob
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from spellchecker import SpellChecker
from datetime import datetime, timedelta
import re,string,math
import gensim
from gensim import corpora, models
from sklearn.preprocessing import LabelEncoder
wnl = WordNetLemmatizer()

In [2]:
def change_dual_words(word):
    first=''
    second=''
    def get_correct_word(w):
        if w=="sha":
            return("shall")
        elif w=="wo":
            return("will")
        elif w=="y":
            return("you")
        else:
            return(w)
#     word=word.replace('$','$').replace('+','')
    word=word.strip().strip(':').strip("'").strip(',').strip('`').strip('[').strip(']').strip('-').strip('_').strip('#').lower()
    word=word.strip('?').strip('+').strip('/').strip('(').strip(')').strip(';').strip('!').strip('.').strip('?')
    if word.endswith("n't"):
        first=get_correct_word(word[:word.index("n't")])
        second="not"
    elif word.endswith("'ll"):
        first=get_correct_word(word[:word.index("'ll")])
        second="will"
    elif word.endswith("'ve"):
        first=get_correct_word(word[:word.index("'ve")])
        second="have"
    elif word.endswith("'re"):
        first=get_correct_word(word[:word.index("'re")])
        second="are"
    elif word.endswith("'d"):
        first=get_correct_word(word[:word.index("'d")])
        second="did"
    elif word.endswith("'s"):
        first=get_correct_word(word[:word.index("'s")])
        second="is"
    elif word.endswith("'all"):
        first=get_correct_word(word[:word.index("'all")])
        second="all"
    else:
        first=word
        second="<allclear>"
    return((first,second))

In [3]:
def cleanLine(line):
    line=line.encode('ascii','ignore').decode(encoding = "ISO-8859-1")
    line=line.replace('&#36;','DOLLAR').replace('+',' ').replace('$','DOLLAR')
    remove_digits=str.maketrans('', '', string.digits)
    line=line.translate(remove_digits)
    
    tokenizer = RegexpTokenizer('\w+-+\w+')
    hyphenated_word=tokenizer.tokenize(line)
    for word in hyphenated_word:
        line=line.replace(word,word.replace('-',' '))
        
    tokenizer = RegexpTokenizer('\w+,+\w+')
    hyphenated_word=tokenizer.tokenize(line)
    for word in hyphenated_word:
        line=line.replace(word,word.replace(',',' '))
        
        
    tokenizer = RegexpTokenizer('\D+/+\D+')
    hyphenated_word=tokenizer.tokenize(line)
    for word in hyphenated_word:
        line=line.replace(word,word.replace('/',' '))
        
    table = str.maketrans(dict.fromkeys(string.punctuation))
    line=line.translate(table)
    
#     print(hyphenated_word)
    stopword = stopwords.words('english')
    stopword.append(['aa','aah','aahhs','aahs','(',')',"''",',','``','-','.','4','2','3po', '6x9', '8mm','5','9','26','$$','$@^@','&#','&nbsp','&quot','#@%$^','==','>from','>honk','@#$^%','@#%','@$&%','a&m','a&w','a+'])
#     stopwordList=set(stopword)
    
    word_lst=line.split()
    clean_lst=[]
    for idx in range(0,len(word_lst)):
        if word_lst[idx] not in punctuation and word_lst[idx] not in stopword:
            dual_words=change_dual_words(word_lst[idx])
            if dual_words[1]=='<allclear>':
                    clean_lst.append(dual_words[0])
            else:
                if dual_words[0] not in stopword:
                    clean_lst.append(dual_words[0])
                if dual_words[1] not in stopword:
                    clean_lst.append(dual_words[1])
#     processed_lst.append('</s>')
    line=str(' ').join(clean_lst)
#     line=str(TextBlob(line).correct())
    return(line)

In [4]:
def get_relevance_date(dt):
    if dt.weekday()==5:
        return(dt - timedelta(1))
    elif dt.weekday()==6:
        return(dt - timedelta(2))
    else:
        return(dt)

In [14]:
def Check_Dates(df,header):
    print('Counts for '+header)
    print('Required no. of Days: ',(df.Date.max()-df.Date.min()).days,\
          'Existing no. of Days: ',len(df.Date.unique()),\
          'Missing no. Of Days: ',(df.Date.max()-df.Date.min()).days-len(df.Date.unique()))

In [15]:
to_datetime = lambda d: datetime.strptime(d, "%d/%m/%Y")
ft_data=pd.read_csv('../Data/NewsData/financial_times.csv',sep='|',names=['Date', 'HeadLines'],converters={'Date': to_datetime},encoding = "ISO-8859-1")
mc_data=pd.read_csv('../Data/NewsData/moneycontrol.csv',sep='|',names=['Date', 'HeadLines'],converters={'Date': to_datetime},encoding = "ISO-8859-1")
op_data=pd.read_csv('../Data/NewsData/oilprice.csv',sep='|',names=['Date', 'HeadLines'],converters={'Date': to_datetime},encoding = "ISO-8859-1")
inv_data=pd.read_csv('../Data/NewsData/investing.csv',sep='|',names=['Date', 'HeadLines'],converters={'Date': to_datetime},encoding = "ISO-8859-1")
mc_data.drop(index=mc_data.iloc[mc_data.HeadLines.isin(['Commodities@moneycontrol \\','Commodities@Moneycontrol \\','Editor\'s Take \\','Podcast \\']).values].index,axis=0,inplace=True)
data = pd.concat([ft_data,mc_data,op_data,inv_data],axis=0,sort=False,ignore_index=True)
data=data.drop_duplicates()
Check_Dates(ft_data,'Financial Times')
Check_Dates(mc_data,'Money Control')
Check_Dates(op_data,'Oilprice.com')
Check_Dates(inv_data,'Investing.com')
Check_Dates(data,'Combined')
# print((op_data.Date.max()-op_data.Date.min()).days,len(op_data.Date.unique()),(op_data.Date.max()-op_data.Date.min()).days-len(op_data.Date.unique()))
# print((mc_data.Date.max()-mc_data.Date.min()).days,len(mc_data.Date.unique()),(mc_data.Date.max()-mc_data.Date.min()).days-len(mc_data.Date.unique()))
# print((ft_data.Date.max()-ft_data.Date.min()).days,len(ft_data.Date.unique()),(ft_data.Date.max()-ft_data.Date.min()).days-len(ft_data.Date.unique()))
# print((inv_data.Date.max()-inv_data.Date.min()).days,len(inv_data.Date.unique()),(inv_data.Date.max()-inv_data.Date.min()).days-len(inv_data.Date.unique()))
# print((data.Date.max()-data.Date.min()).days,len(data.Date.unique()),(data.Date.max()-data.Date.min()).days-len(data.Date.unique()))

Counts for Financial Times
Required no. of Days:  1467 Existing no. of Days:  1285 Missing no. Of Days:  182
Counts for Money Control
Required no. of Days:  3323 Existing no. of Days:  2643 Missing no. Of Days:  680
Counts for Oilprice.com
Required no. of Days:  3841 Existing no. of Days:  1768 Missing no. Of Days:  2073
Counts for Investing.com
Required no. of Days:  3365 Existing no. of Days:  1762 Missing no. Of Days:  1603
Counts for Combined
Required no. of Days:  3851 Existing no. of Days:  3074 Missing no. Of Days:  777


In [7]:
to_datetime = lambda d: datetime.strptime(d, "%Y-%m-%d")
calc_movement= lambda mv: 'down' if mv<0 else ('up' if mv>0 else 'nomovement')
data['Agg_Col']=data.Date.apply(lambda dt:get_relevance_date(dt))
price_data=pd.read_csv('../Data/NewsData/CrudePriceSmoothened.csv',sep='|',converters={'Date': to_datetime},encoding = "ISO-8859-1")
# price_data['NEXT_DAY']=price_data.sort_values('DATE',ascending=False,kind='mergesort')['CLOSING_PRICE'].shift(1)
# price_data['DIFF']=price_data.sort_values('DATE',ascending=False,kind='mergesort')['CLOSING_PRICE'].shift(1)-price_data['CLOSING_PRICE']
price_data['MoveMent']=(price_data.sort_values('Date',ascending=False,kind='mergesort')['CP_trend'].shift(1)-price_data['CP_trend']).apply(calc_movement)
data=data.join(price_data[['Date','MoveMent','CP_trend']].set_index('Date'), on='Agg_Col',how='inner')

In [8]:
data['Cleaned_HL']=data.HeadLines.apply(lambda line:cleanLine(line))
data['Lemmetized_HL']=data.Cleaned_HL.str.split().apply(lambda word_lst: str(' ').join([wnl.lemmatize(word) for word in word_lst]))
processed_df=data.Cleaned_HL.apply(lambda line: TextBlob(line).sentiment).apply(pd.Series)
processed_df.columns=['Polarity','Subjectivity']
data=pd.concat([data,processed_df],axis=1,sort=False)
# processed_df['Date']=data.Date
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer=TfidfVectorizer(use_idf=True,stop_words=['aa','aaa'])
# fitted_vec=vectorizer.fit_transform(data.Lemmetized_HL)
# tf_idf_df=pd.DataFrame(fitted_vec.todense(),columns=vectorizer.get_feature_names())
# processed_df = pd.concat([processed_df,tf_idf_df],axis=1,sort=False)
data[['Date','Cleaned_HL','MoveMent']].to_csv('../Data/Processed_Data/input_data_for_cnn.csv',sep='|',index=False)
data[['Date','Agg_Col','Lemmetized_HL','MoveMent','Polarity','Subjectivity']].to_csv('../Data/Processed_Data/input_data_for_lda.csv',sep='|',index=False)

In [82]:
from collections import Counter
jd=pd.concat([ft_data,op_data],axis=0,sort=False,ignore_index=True)
jd['Cleaned_HL']=jd.HeadLines.apply(lambda line:cleanLine(line))
stopword = stopwords.words('english')
stopword.extend(['says','fall','back','hit','high','since','low','set','profits','first','higher','despite','rally','drop',
                 'record','boost','growth''could','new','cut','demand','boom','exports','industry','cuts','count','global',
                 'may','year','major','north','prices','production','big','market','markets','price','world','worlds',
                 'output','deal','next','inventories','time','majors','ahead','rig','lower','hits','groups','biggest','ipo',
                 'group','asia','falls','sector','offshore','amid','shares','years','chief','future','warns','trade','stake',
                 'investment','companies','sell','recovery','draw','profit','war','month','gains','rises','total','traders',
                 'jump','buy','end','slide','rebound','plans','decline','inventory','climate','risk','build','trading',
                 'minister','assets','exxon','fund','still','meeting','th','billion','forecast','report','business','data',
                 'east','rising','day','plan','looks','debt','company','sands','field','reports','export','change','crisis',
                 'fears','earnings','takes','review','loss','faces','tax','premier','another','dollarm','funds','raises',
                 'million','eyes','imports','top','sale','sees','threat','drops','last','talks','boosts','pressure','seeks',
                 'face','exxonmobil','spending','largest','look','pemex','bond','two','one','opening','bullish','much',
                 'take','keep','tanker','forecasts','live','project','keystone','vitol','extends','pay','middle','commodity',
                 'hedge','highest','power','continues','push','commodities','quote','long','q','hopes','india','south',
                 'eni','expected','key','threatens','green','slump','latest','strong','worst','best','soon','needs','go',
                 'well','potential','five','outlook','state','play','losses','near','trader','increase','bid','wall','move',
                 'bets','economic','raise','banks','second','wood','makes','sp','rules','share','trumps','quarter',
                 'trafigura','producer','wont','lead','slips','europe','minute','need','three','exploration','become',
                 'falling','start','bhp','april','see','goldman','make','close','attack','takeover','ban','non','sells',
                 'march','economy','crash','attacks','discovery','continue','bakken','way','slides','break','poised',
                 'services','third','concerns','watch','target','battle','shells','focus','case','cost','ceo','level','deals',
                 'huge','post','freeze','bet','jumps','calls','find','must','climbs','equities','coming','plunge','get',
                 'costs','return','fails','good','help','worries','targets','gain','genel','dividend','head','weekly','behind',
                 'shift','stream','extend','really','january','hurricane','months','point','signs','strategy','pushes',
                 'offer','ready','agree','glencore','far','tough','corruption','part','centrica','deepwater','winners','cuadrilla',
                 'slip','capacity','policy','cheap','foreign','strike','turn','sends','unit','eye','coast','fire','storm',
                 'game','cash','lifts','tensions','step','anadarko','shipping','agrees','cant','cap','projects','wins',
                 'fed','bn','holds','harvey','america','short','support','deeper','lowest','bonds','junk','schlumberger',
                 'aims','weak','surprise','november','straight','double','rout','term','nord','xl','expect','slowdown',
                 'puts','halliburton','eu','swings','buys','hold','strikes','dip','climb','drive','investor','might',
                 'national','capital','election','money','fresh','weeks','gold','giant','questions','away','loses','debate',
                 'discoveries','extension','turns','taking','struggle','signals','rates','wealth','st','decision','launches',
                 'risks','impact','returns','fight','brexit','bearish','half','volatility','dakota','beyond','independence',
                 'enquest','supplies','american','futures','track','july','rallies','forces','opportunity','occidental',
                 'meet','government','towards','sales','lose','moves','real','going','rush','stock','small','numbers',
                 'street','geopolitical','cairn','bust','bank','africa','amec','per','less','stockpiles','fields','nuclear',
                 'sets','life','conocophillips','pipelines','chairman','aramcos','cutting','enough','june','pump','come',
                 'british','strategic','slow','work','political','rigs','almost','decade','add','brazils','rebounds','west'
                 'shows','whats','fast','february','obama','abu','norwegian','dont','struggles','spot','balance','dispute',
                 'finds','tops','collapse','court','auction','bear','imf','probe','beat','net','analysts','action','open',
                 'never','putin','compliance','angola','dutch','streak','value','president','highs','claims','longer','large',
                 'hard','weaker','significant','great','call','keeps','lift','surges','london','land','ends','looking','grow',
                 'breaks','massive','losers','storage','right','trillion','revolution','santos','becomes','posts','boss',
                 'listing','boosted','alberta','september','even','challenge','recover','alliance','isnt','blow','tech',
                 'steady','remain','yet','stop','jobs','test','black','west','gives','volatile','rights','talk','making',
                 'sign','slumps','force','problems','transition','royal','miss','revenues','expectations','moodys','bounce',
                 'finally','offers','currencies','corporate','curb','asset','retreat','bigger','dilemma','opens','cartel',
                 'reach','august','crunch','soars','better','sharply','bulls','shake','urges','stay','doha','backs','private',
                 'buying','equity','optimism','credit','drill','leads','ineos','kirkuk','losing','halt','ft','left','ease',
                 'without','weighs','yen','former','hope','give','gets','maersk','launch','woes','results','put','pre',
                 'resources','spike','points','role','tuesday','ups','cars','run','begin','international','would','financial',
                 'four','heavy','clean','save','contract','japan','era','security','mark','issue','vows','reliance','invest',
                 'bps','reforms','survive','york','hotspot','budget','vote','levels','legal','win','nears','board','announces',
                 'things','remains','patch','likely','reality','ghana','free','dollartn','halts','comeback','warning','venture',
                 'goes','comes','frontier','problem','baker','gunvor','doubles','flow','december','pain','hughes','firstft'])
stopwordList=set(tuple(stopword))
word_list=jd.Cleaned_HL.str.split().explode().reset_index().set_index('Cleaned_HL').drop(stopwordList,errors=None).reset_index().drop('index',axis=1)
word_list=word_list.groupby('Cleaned_HL')['Cleaned_HL'].count()\
        .rename(columns={'Cleaned_HL':'Word'})\
        .reset_index()
word_list.columns=['word','cnt']
word_list.sort_values('cnt',ascending=False,inplace=True)
'|'.join(word_list[word_list.cnt>=15].head(150).word.to_list())

'oil|us|opec|saudi|crude|energy|shale|gas|dollar|iran|could|stocks|dollarbn|premium|russia|week|arabia|bp|supply|china|sea|aramco|rise|peak|shell|trump|pipeline|uk|sanctions|brent|venezuela|investors|growth|producers|iea|nigeria|drilling|gulf|rosneft|russian|saudis|barrel|iraq|fracking|bpd|glut|fuel|mexico|eia|chevron|chinas|libya|opecs|iranian|permian|canadian|chinese|reserves|arctic|petrobras|gasoline|russias|texas|oils|tullow|lng|surge|venezuelas|canadas|canada|barrels|oilfield|mexicos|irans|asian|natural|carbon|arabias|norway|drillers|european|fossil|libyan|gazprom|fuels|iraqi|refiners|norways|petroleum|tankers|nigerias|libyas|wti|venezuelan|emissions|repsol|statoil|brazil|mexican|electric|refinery|kurdistan|nigerian|australian|refining|petrol|shows|iraqs|kuwait|syria|kurdish|kazakhstan|coal|turkey|refineries|uae|petrofac|lukoil'

In [80]:
# [x for x in word_list[word_list.cnt<=17].head(150).word]

# word_list[word_list.cnt<=17].head(50)
word_list[word_list.cnt>=15]

,word,cnt
3906,oil,5176
6268,us,1177
3937,opec,696
5015,saudi,564
1315,crude,541
...,...,...
6091,turkey,17
1062,coal,17
6116,uae,16
3450,lukoil,15


In [10]:
data.shape

(15858, 10)